In [3]:
a = ['gtfs_date', 'dayofweek', 'hour', 'gtfs_route_id', 'gtfs_direction_id', 'stop_id', 'transit_date', 'trip_id', 'day', 'overload_id', 'vehicle_id', 'block_abbr', 'activation_date', 'activation_date_str', 'arrival_time', 'arrival_time_str', 'block_stop_order', 'deactivation_date', 'deactivation_date_str', 'departure_time', 'departure_time_str', 'load', 'load_factor', 'map_latitude', 'map_longitude', 'offs', 'ons', 'pattern_num', 'route_direction_name', 'route_id', 'scheduled_time', 'scheduled_time_str', 'source_pattern_id', 'stop_id_list', 'stop_id_original', 'stop_name', 'stop_sequence', 'stop_sequence_list', 'transit_date_str', 'update_date', 'vehicle_capacity', 'zero_load_at_trip_end', 'count', 'darksky_temperature', 'darksky_humidity', 'darksky_nearest_storm_distance', 'darksky_precipitation_intensity', 'darksky_precipitation_probability', 'darksky_pressure', 'darksky_wind_gust', 'darksky_wind_speed', 'weatherbit_rh', 'weatherbit_wind_spd', 'weatherbit_app_temp', 'weatherbit_temp', 'weatherbit_snow', 'weatherbit_precip', 'gtfs_file', 'gtfs_shape_id', 'gtfs_start_date', 'gtfs_end_date', 'gtfs_number_of_scheduled_trips', 'gtfs_number_of_scheduled_trips_at_stop', 'delay_time', 'dwell_time', 'prev_sched', 'sched_hdwy', 'prev_depart', 'actual_hdwy', 'is_gapped', 'is_bunched', 'is_target', 'year', 'month']
[print(_a) for _a in a]

gtfs_date
dayofweek
hour
gtfs_route_id
gtfs_direction_id
stop_id
transit_date
trip_id
day
overload_id
vehicle_id
block_abbr
activation_date
activation_date_str
arrival_time
arrival_time_str
block_stop_order
deactivation_date
deactivation_date_str
departure_time
departure_time_str
load
load_factor
map_latitude
map_longitude
offs
ons
pattern_num
route_direction_name
route_id
scheduled_time
scheduled_time_str
source_pattern_id
stop_id_list
stop_id_original
stop_name
stop_sequence
stop_sequence_list
transit_date_str
update_date
vehicle_capacity
zero_load_at_trip_end
count
darksky_temperature
darksky_humidity
darksky_nearest_storm_distance
darksky_precipitation_intensity
darksky_precipitation_probability
darksky_pressure
darksky_wind_gust
darksky_wind_speed
weatherbit_rh
weatherbit_wind_spd
weatherbit_app_temp
weatherbit_temp
weatherbit_snow
weatherbit_precip
gtfs_file
gtfs_shape_id
gtfs_start_date
gtfs_end_date
gtfs_number_of_scheduled_trips
gtfs_number_of_scheduled_trips_at_stop
delay_tim

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Sampled using scheduled times

In [4]:
import numpy as np
np.random.random()

0.029412943697365357

In [5]:
%matplotlib inline

In [6]:
import os
os.chdir("/home/jptalusan/mta_simulator")

import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import gtfs_kit as gk
import numpy as np
import osmnx as ox
import networkx as nx
from tqdm import tqdm
from shapely.geometry import Polygon, LineString, Point
import warnings
warnings.filterwarnings('ignore')

/home/jptalusan/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [7]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark import SparkConf
import pandas as pd
import pickle
from tqdm import tqdm

In [8]:
spark = SparkSession.builder.config('spark.executor.memory', '2g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '2g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.autoBroadcastJoinThreshold", -1)\
        .config("spark.driver.maxResultSize", 0)\
        .config("spark.shuffle.spill", "true")\
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
        .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
        .config("spark.ui.showConsoleProgress", "false")\
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

22/09/14 09:47:26 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
22/09/14 09:47:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 09:47:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
f = os.path.join('/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
apcdata.createOrReplaceTempView("apc")
apcdata = spark.sql("SELECT * FROM apc LIMIT 1")

In [16]:
apcdata.show(1, vertical=True)

-RECORD 0------------------------------------------------------
 gtfs_date                              | 2021-10-04 00:00:00  
 dayofweek                              | 4                    
 hour                                   | null                 
 gtfs_route_id                          | 4                    
 gtfs_direction_id                      | 0                    
 stop_id                                | SHE7SEN              
 transit_date                           | 2021-12-01 00:00:00  
 trip_id                                | 261607               
 day                                    | 1                    
 overload_id                            | 0                    
 vehicle_id                             | 732                  
 block_abbr                             | 410                  
 activation_date                        | 2021-11-18 00:00:00  
 activation_date_str                    | 11/18/21             
 arrival_time                           

In [14]:
apcdata.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 74 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   gtfs_date                               1 non-null      datetime64[ns]
 1   dayofweek                               1 non-null      int32         
 2   hour                                    0 non-null      float64       
 3   gtfs_route_id                           1 non-null      object        
 4   gtfs_direction_id                       1 non-null      int64         
 5   stop_id                                 1 non-null      object        
 6   transit_date                            1 non-null      datetime64[ns]
 7   trip_id                                 1 non-null      object        
 8   day                                     1 non-null      int32         
 9   overload_id                             1 non-null      in

In [20]:
f = os.path.join('/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
print(apcdata.columns)
get_columns = ['trip_id', 'transit_date', 'arrival_time', 'block_abbr', 'scheduled_time', 'vehicle_id',
              'stop_sequence', 'stop_id_original', 'load', 'ons', 'route_id', 'route_direction_name']
get_str = ", ".join([c for c in get_columns])
apcdata.createOrReplaceTempView("apc")
# # filter subset
query = f"""
       SELECT {get_str}
       FROM apc
       """
print(query)
apcdata = spark.sql(query)
apcdata = apcdata.withColumn("route_id_direction", F.concat_ws('_',apcdata.route_id, apcdata.route_direction_name))
apcdata = apcdata.drop('route_id', 'route_direction_name')
apcdf = apcdata.toPandas()
fp = os.path.join('/home/jptalusan/mta_simulator/code/data/simulator_baseline2.pkl')
apcdf.to_pickle(fp)

['gtfs_date', 'dayofweek', 'hour', 'gtfs_route_id', 'gtfs_direction_id', 'stop_id', 'transit_date', 'trip_id', 'day', 'overload_id', 'vehicle_id', 'block_abbr', 'activation_date', 'activation_date_str', 'arrival_time', 'arrival_time_str', 'block_stop_order', 'deactivation_date', 'deactivation_date_str', 'departure_time', 'departure_time_str', 'load', 'load_factor', 'map_latitude', 'map_longitude', 'offs', 'ons', 'pattern_num', 'route_direction_name', 'route_id', 'scheduled_time', 'scheduled_time_str', 'source_pattern_id', 'stop_id_list', 'stop_id_original', 'stop_name', 'stop_sequence', 'stop_sequence_list', 'transit_date_str', 'update_date', 'vehicle_capacity', 'zero_load_at_trip_end', 'count', 'darksky_temperature', 'darksky_humidity', 'darksky_nearest_storm_distance', 'darksky_precipitation_intensity', 'darksky_precipitation_probability', 'darksky_pressure', 'darksky_wind_gust', 'darksky_wind_speed', 'weatherbit_rh', 'weatherbit_wind_spd', 'weatherbit_app_temp', 'weatherbit_temp', '

In [22]:
apcdf

,trip_id,transit_date,arrival_time,block_abbr,scheduled_time,vehicle_id,stop_sequence,stop_id_original,load,ons,route_id_direction
0,261607,2021-12-01,NaT,410,2021-12-01 06:41:59,732,9,SHE7SEN,-1.0,0.0,4_FROM DOWNTOWN
1,261608,2021-12-01,NaT,410,2021-12-01 07:21:59,732,9,SHE7SEN,2.0,0.0,4_FROM DOWNTOWN
2,261609,2021-12-01,NaT,410,2021-12-01 08:01:59,732,9,SHE7SEN,0.0,0.0,4_FROM DOWNTOWN
3,261610,2021-12-01,NaT,410,2021-12-01 08:41:59,732,9,SHE7SEN,4.0,0.0,4_FROM DOWNTOWN
4,261611,2021-12-01,NaT,410,2021-12-01 09:21:59,732,9,SHE7SEN,5.0,0.0,4_FROM DOWNTOWN
...,...,...,...,...,...,...,...,...,...,...,...
22617696,218633,2020-04-29,2020-04-29 19:49:02,4200,2020-04-29 19:49:02,1904,22,8AJEFSM,1.0,0.0,42_TO DOWNTOWN
22617697,219046,2020-04-29,2020-04-29 12:23:58,5604,2020-04-29 12:19:27,1819,33,GALCONNN,4.0,0.0,56_FROM DOWNTOWN
22617698,219052,2020-04-29,2020-04-29 14:19:32,5604,2020-04-29 14:19:27,1819,33,GALCONNN,6.0,0.0,56_FROM DOWNTOWN
22617699,219058,2020-04-29,2020-04-29 16:19:18,5604,2020-04-29 16:19:27,1819,33,GALCONNN,9.0,0.0,56_FROM DOWNTOWN


# Simulator

## Setup map
* Match locations with vertices on the map

In [ ]:
fp = os.path.join('code/data', 'shapefiles', "tncounty")
gdf_county = gpd.read_file(fp)
gdf_dav = gdf_county[gdf_county["NAME"] == "Davidson"]
gdf_dav = gdf_dav.to_crs("EPSG:4326")

G = ox.graph_from_polygon(gdf_dav.geometry.iloc[0], network_type='drive')
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

fp = os.path.join('code/data', 'davidson_graph.graphml')
ox.save_graphml(G, fp)

In [ ]:
f = os.path.join('/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
apcdata.columns
get_columns = ['stop_sequence', 'stop_id_original', 'stop_name', 'map_latitude', 'map_longitude']
get_str = ", ".join([c for c in get_columns])
apcdata.createOrReplaceTempView("apc")

# # filter subset
query = f"""
SELECT {get_str}
FROM apc
"""
apcdata = spark.sql(query)
apcdata = apcdata.drop_duplicates(['stop_id_original'])
apcdf = apcdata.toPandas()

apcdf['nearest_node'] = ox.nearest_nodes(G, apcdf['map_longitude'], apcdf['map_latitude'])
apcdf['nearest_edge'] = ox.nearest_edges(G, apcdf['map_longitude'], apcdf['map_latitude'])

In [ ]:
fp = os.path.join('code/data', 'stops_node_matching.pkl')
apcdf.to_pickle(fp)

In [ ]:
apcdf

In [ ]:
r = ox.shortest_path(G, 202177835, 9702091174, weight='travel_time')
ox.plot_graph_route(G, r)

In [ ]:
cn = apcdf[apcdf['stop_id_original'] == 'MCC4_24']['nearest_node'].values[0]
nn = apcdf[apcdf['stop_id_original'] == 'UNI2AEF']['nearest_node'].values[0]
r = ox.shortest_path(G, cn, nn, weight='travel_time')
cols = ['osmid', 'length', 'travel_time']
attrs = ox.utils_graph.get_route_edge_attributes(G, r)
tt = pd.DataFrame(attrs)[cols]['travel_time'].sum()
tt

In [ ]:
pd.DataFrame(attrs)

In [ ]:

fp = os.path.join('code/Scenarios/data/sample_trip_plan.pkl')
with open(fp, 'rb') as f:
    overall_trip_plan = pickle.load(f)

In [ ]:
overall_trip_plan[0]['nearest_node'] = overall_trip_plan[0]['stop_id'].apply(lambda x: apcdf[apcdf['stop_id_original'] == x]['nearest_node'].values[0])
overall_trip_plan[0]['next_node'] = overall_trip_plan[0]['nearest_node'].shift(-1)
overall_trip_plan[0] = overall_trip_plan[0].dropna(subset=['next_node'])

def get_travel_time(cn, nn):
    try:
        r = ox.shortest_path(G, cn, nn, weight='travel_time')
        cols = ['osmid', 'length', 'travel_time']
        attrs = ox.utils_graph.get_route_edge_attributes(G, r)
        tt = pd.DataFrame(attrs)[cols]['travel_time'].sum()
        
        return tt
    except:
        return -1

overall_trip_plan[0]['r'] = overall_trip_plan[0].apply(lambda x: get_travel_time(x['nearest_node'], x['next_node']), axis=1)

In [ ]:
overall_trip_plan[0]

## Convert APC load data to something static and fast to access (look up)

In [62]:
import pandas as pd
fp = os.path.join('/home/jptalusan/mta_simulator/code/data/simulator_baseline2.pkl')
baseline_data = pd.read_pickle(fp)
baseline_data['dow'] = baseline_data['scheduled_time'].dt.dayofweek
baseline_data['IsWeekend'] = (baseline_data["scheduled_time"].dt.weekday >= 5).astype('int')
baseline_data['time'] = baseline_data['scheduled_time'].dt.time
baseline_data = baseline_data.query("load >= 0 and load <= 100")
baseline_data = baseline_data.groupby(['route_id_direction', 'block_abbr', 'stop_id_original', 'time', 'IsWeekend']).agg({'load':list})
fp = os.path.join('/home/jptalusan/mta_simulator/code/data/loads_by_scheduled_time.pkl')
baseline_data.to_pickle(fp)

## For boarding
* Aggregate in 1 hour windows then group like occupancy

In [63]:
import pandas as pd
fp = os.path.join('/home/jptalusan/mta_simulator/code/data/simulator_baseline2.pkl')
baseline_data = pd.read_pickle(fp)
baseline_data['dow'] = baseline_data['scheduled_time'].dt.dayofweek
baseline_data['IsWeekend'] = (baseline_data["scheduled_time"].dt.weekday >= 5).astype('int')
baseline_data['time'] = baseline_data['scheduled_time'].dt.time
baseline_data = baseline_data.query("ons >= 0 and ons <= 100")
baseline_data = baseline_data.groupby(['route_id_direction', 'block_abbr', 'stop_id_original', 'time', 'IsWeekend']).agg({'ons':list})
fp = os.path.join('/home/jptalusan/mta_simulator/code/data/ons_by_scheduled_time.pkl')
baseline_data.to_pickle(fp)

In [64]:
baseline_data

ons
route_id_direction block_abbr stop_id_original time     IsWeekend                                                   
14_FROM DOWNTOWN   1400       1SJAMNM          06:18:13 0          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                                        1          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                               06:18:41 0          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                               06:21:13 0          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                                        1          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
...                                                                                                              ...
9_TO DOWNTOWN      8600       3AMONSN          07:46:39 0          [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...
                              3AVANSF          07:45:45 0          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              DOMFREEN         07:41:00 0          [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              MAD2AVEN         07:47:42 0          [2.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...
                              MCC5_2           07:54:00 0          [0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...

[450572 rows x 1 columns]

## For Travel Times
* for each route_id_dir, block, stop to stop

In [70]:
fp = os.path.join('code/data/simulator_baseline2.pkl')
baseline_data = pd.read_pickle(fp).dropna(subset=['arrival_time']).sort_values(by=['transit_date', 'trip_id', 'stop_sequence'])
baseline_data['dow'] = baseline_data['scheduled_time'].dt.dayofweek
baseline_data['IsWeekend'] = (baseline_data["scheduled_time"].dt.weekday >= 5).astype('int')
baseline_data['time'] = baseline_data['scheduled_time'].dt.time
df_arr = []

from multiprocessing import Pool, cpu_count
import datetime as dt

def get_traveltimes(tdf):
    tdf = tdf.sort_values('stop_sequence')
    tdf['scheduled_timestamp'] = (tdf['arrival_time'] - dt.datetime(1970,1,1)).dt.total_seconds()
    tdf['time_to_next_stop'] = tdf['scheduled_timestamp'].shift(-1) - tdf['scheduled_timestamp']
    tdf['time_to_next_stop'] = tdf[tdf['time_to_next_stop'] >= 0]
    tdf = tdf.drop('scheduled_timestamp', axis=1)
    tdf = tdf.fillna(0)
    return tdf
    
def applyParallel(dfGrouped, func):
    with Pool(cpu_count()) as p:
        ret_list = p.map(func, [group for name, group in dfGrouped])
    return pd.concat(ret_list)

out_arr = applyParallel(baseline_data.groupby(['block_abbr', 'route_id_direction', 'trip_id', 'transit_date']), get_traveltimes)
tdf = out_arr.groupby(['route_id_direction', 'block_abbr', 'stop_sequence', 'stop_id_original', 'time', 'IsWeekend']).agg({'time_to_next_stop':list})
fp = os.path.join('/home/jptalusan/mta_simulator/code/data/travel_time_by_scheduled_time.pkl')
tdf.to_pickle(fp)

In [72]:
tdf

time_to_next_stop
route_id_direction block_abbr stop_sequence stop_id_original time     IsWeekend                                                   
14_FROM DOWNTOWN   1400       1             MCC4_20          06:15:00 0          [339.0, 476.0, 605.0, 861.0, 1010.0, 888.0, 90...
                                                                      1          [307.0, 351.0, 488.0, 378.0, 231.0, 363.0, 266...
                                                             06:18:00 0          [328.0, 308.0, -239.0, 562.0, 332.0, 658.0, 68...
                                                                      1          [821.0, 398.0, 458.0, 764.0, 276.0, 585.0, 526...
                                                             07:15:00 0          [519.0, 572.0, 918.0, 520.0, 628.0, 850.0, 852...
...                                                                                                                            ...
9_TO DOWNTOWN      8600       6             2AVJUNSF         07:48:54 0          [26.0, 30.0, 42.0, 24.0, 24.0, 40.0, 24.0, 22....
                                                             07:48:57 0                                         [34.0, 58.0, 34.0]
                              7             2ASTOSM          07:50:02 0          [228.0, 242.0, 169.0, 188.0, 284.0, 208.0, 266...
                                                             07:50:06 0                                      [262.0, 328.0, 197.0]
                              8             MCC5_2           07:54:00 0          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

[533906 rows x 1 columns]

In [167]:
tdf.loc[('23_FROM DOWNTOWN', 2311, 8, 'VAIBRIEM',)]

time_to_next_stop
time_window IsWeekend                                                   
18          1             [10.0, 10.0, 10.0, 8.0, 8.0, 10.0, 10.0, 12.0]
19          0          [10.0, 12.0, 10.0, 12.0, 10.0, 12.0, 8.0, 12.0...
22          1                   [10.0, 10.0, 8.0, 10.0, 10.0, 8.0, 10.0]
23          0                                   [10.0, 12.0, 12.0, 14.0]
            1                 [10.0, 10.0, 12.0, 10.0, 10.0, 10.0, 14.0]

In [165]:
adf = tdf.loc[('23_FROM DOWNTOWN', 2311)]
adf = adf.query('IsWeekend == 0')
# a = 26
# b = 48
# # adf.query('time_window == @a or time_window == @b').sample(1)['time_to_next_stop'].values[0]
# adf = adf.explode('time_to_next_stop').query('time_to_next_stop > 0').reset_index()
adf = adf.explode('time_to_next_stop').reset_index()
# adf[adf['time_window'].isin(range(22, 32))].sample(1)

adf

,stop_sequence,stop_id_original,time_window,IsWeekend,time_to_next_stop
0,1,DOVDICWF,18,0,644.0
1,1,DOVDICWF,18,0,509.0
2,1,DOVDICWF,18,0,637.0
3,1,DOVDICWF,18,0,268.0
4,1,DOVDICWF,18,0,391.0
...,...,...,...,...,...
110128,70,DWMRT,76,0,0.0
110129,70,DWMRT,76,0,0.0
110130,70,DWMRT,76,0,0.0
110131,70,DWMRT,81,0,0.0


In [111]:
list(range(0, 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [58]:
len(tdf.loc[('3_TO DOWNTOWN', 300, 1, 'WHICHASF', 45, 0)]['time_to_next_stop'])

31

## Generate travel distance pairs for all stops

In [171]:
fp = '/media/seconddrive/wego-occupancy-JP/data/static_gtfs/WeGoRawGTFS/04-october-2021-fixed.zip'
feed = gk.read_feed(fp, dist_units='km')
feed.validate()
stop_times_df = gk.get_stop_times(feed)
stop_pairs = []
for trip_id, trip_df in stop_times_df.groupby('trip_id'):
    trip_df['next_stop_id'] = trip_df['stop_id'].shift(-1)
    trip_df = trip_df.fillna(0)
    trip_df['shape_dist_traveled_km'] = (trip_df['shape_dist_traveled'].shift(-1) - trip_df['shape_dist_traveled'])
    trip_df = trip_df[['stop_id', 'next_stop_id', 'shape_dist_traveled_km']][:-1]
    stop_pairs.append(trip_df)
stop_pairs = pd.concat(stop_pairs)
stop_pairs = stop_pairs.drop_duplicates()
stop_pairs.head()

,stop_id,next_stop_id,shape_dist_traveled_km
0,MCC4_20,UNI2AEF,0.5776
1,UNI2AEF,1SWOONM,0.6813
2,1SWOONM,1SJAMNM,0.3601
3,1SJAMNM,N1SOLDNM,0.3169
4,N1SOLDNM,DICGRANN,0.8610


In [172]:
fp = os.path.join('code/Scenarios/data/gtfs_distance_pairs_km.pkl')
stop_pairs.to_pickle(fp)

## Generate Disruption probabilities

In [ ]:
## Generate Disruption probabilities
# Get service disruption dataset
fp = os.path.join('code/data/Service Disruptions_07_2019_08_2022.csv')
disruptions_df = pd.read_csv(fp)
disruptions_df.head()
disruptions_df['DATETIME'] = disruptions_df['DATE'] + ' ' + disruptions_df['TIME']
disruptions_df['DATE'] = pd.to_datetime(disruptions_df['DATE'], format='%m/%d/%y', errors='coerce')
disruptions_df['TIME'] = pd.to_datetime(disruptions_df['TIME'], format='%H:%M:%S', errors='coerce')
disruptions_df['DATETIME'] = pd.to_datetime(disruptions_df['DATETIME'], format='%m/%d/%y %H:%M:%S', errors='coerce')

# Remove weather related disruptions
# disruptions_df = disruptions_df[(disruptions_df['REASON'] != 'Weather')].sort_values(by=['DATETIME']).reset_index(drop=True)
print('Shape:', disruptions_df.shape)
# disruptions_df = disruptions_df.drop(columns=['COMMENTS'])
disruptions_df['BLOCK'] = disruptions_df['BLOCK'].astype('int32')

# Convert to spark dataframe for merging
# disruptions_sp = spark.createDataFrame(disruptions_df)
# disruptions_sp = disruptions_sp.withColumn("BLOCK", F.col("BLOCK").cast(IntegerType()))
disruptions_counts_df = disruptions_df.groupby('START_STOP_ABBR').agg('count')[['REASON']].reset_index()
disruptions_counts_df.sort_values('REASON')
# Count the number of trips throughout this time
start_date = disruptions_df.sort_values(by=['DATETIME']).iloc[0]['DATETIME']
end_date   = disruptions_df.sort_values(by=['DATETIME']).iloc[-1]['DATETIME']
start_date, end_date

# # filter subset
get_columns = ['transit_date', 'trip_id', 'departure_time', 'stop_id_original']
get_str = ", ".join([c for c in get_columns])

query = f"""
SELECT {get_str}
FROM apc
WHERE (transit_date >= '{start_date.date()}') AND (transit_date <= '{end_date.date()}')
"""
print(query)

apcdataafternegdelete = spark.sql(query)
apcdataafternegdelete = apcdataafternegdelete.dropna()
trips_df = apcdataafternegdelete.toPandas()
trips_df = trips_df.groupby('stop_id_original').agg('count').sort_values('trip_id').reset_index()

Shape: (5066, 10)


In [ ]:
merged_df = pd.merge(trips_df, disruptions_counts_df[['START_STOP_ABBR', 'REASON']], left_on='stop_id_original', right_on='START_STOP_ABBR')
all_stop_probabilities = trips_df[['stop_id_original']]
all_stop_probabilities = pd.merge(all_stop_probabilities, merged_df[['stop_id_original', 'probability']], on='stop_id_original', how='outer').fillna(0)
all_stop_probabilities.sort_values('probability')

fp = os.path.join('code/Scenarios/data/disruption_probabilities.pkl')
all_stop_probabilities.to_pickle(fp)

merged_df['probability'] = merged_df['REASON'] / merged_df['transit_date']
merged_df['probability'] = merged_df['probability']/merged_df['probability'].max()
merged_df.sort_values('probability').tail(10)